In [1]:
import pandas as pd
import numpy as np
import tarfile
import io
import glob
import dask.dataframe as dd


import lightgbm as lgb
import warnings
warnings.simplefilter("ignore")

import myFunc as mf

tar = tarfile.open('../data/raw/20181120_track_features.tar.gz', 'r:gz')
csv_files = tar.getnames()

tf_df_list = []

for csv_file in [csv_files[2], csv_files[4]]:
    csv_contents = tar.extractfile(csv_file).read()
    tf_df_list.append(pd.read_csv(io.BytesIO(csv_contents), encoding='utf8'))

tf_df = pd.concat(tf_df_list, ignore_index=True)
tf_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)

kmean300_df = pd.read_csv('../data/interim/all_data/mbKMeans300clusters.csv', usecols=['track_id','clus'])
kmean300_df.rename(columns={'track_id':'track_id_clean'}, inplace=True)



In [2]:
# load the 5 parameter sets with the best performance

import json

list_bayesOpt = glob.glob('../models/SVD/LightGBM_BayesOpt_dec17/for20180918/logs_shuffle10.json')

opt_best_df = pd.DataFrame()
for jsonFile in list_bayesOpt:
    with open(jsonFile) as f:
        optList = []
        for jsonObj in f:
            optDict = json.loads(jsonObj)
            optList.append(optDict)
        
        opt_df = pd.DataFrame(optList)
        opt_df = pd.concat([opt_df.drop(['params'], axis=1), opt_df['params'].apply(pd.Series)], axis=1)
        opt_best_df = pd.concat([opt_best_df,opt_df.sort_values('target',ascending=False).iloc[0:5]])

opt_best_df.sort_values('target',ascending=False)


,target,datetime,bagging_fraction,bagging_freq,batch_size,feature_fraction,learning_rate,min_gain_to_split,nFile,num_iterations,num_leaves
20,0.720543,"{'datetime': '2022-12-20 20:47:08', 'elapsed':...",0.8,2.0,10.0,0.731797,0.153480,0.089333,10.0,850.978738,25.156269
33,0.720360,"{'datetime': '2022-12-21 03:42:12', 'elapsed':...",0.8,2.0,10.0,0.435503,0.226245,0.088026,10.0,859.312532,29.996526
24,0.720333,"{'datetime': '2022-12-20 22:40:18', 'elapsed':...",0.8,2.0,10.0,0.798139,0.153018,0.028336,10.0,842.720473,25.606681
15,0.720302,"{'datetime': '2022-12-20 18:55:13', 'elapsed':...",0.8,2.0,10.0,0.673718,0.346856,0.013443,10.0,842.212623,27.897216
27,0.720035,"{'datetime': '2022-12-21 00:07:55', 'elapsed':...",0.8,2.0,10.0,0.728087,0.355348,0.078096,10.0,858.867160,27.280231


In [3]:
import random

file_list = []

temp_list = []
for logN in range(10):
    temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180917_000000000000.csv')

random.Random(23).shuffle(temp_list)
file_list += temp_list



temp_list = []
for logN in range(10):
    temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180916_000000000000.csv')

random.Random(23).shuffle(temp_list)
file_list += temp_list



temp_list = []
for logN in range(10):
    temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180915_000000000000.csv')

random.Random(23).shuffle(temp_list)
file_list += temp_list



# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180914_000000000000.csv')

# random.Random(23).shuffle(temp_list)
# file_list += temp_list



# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180913_000000000000.csv')

# random.Random(23).shuffle(temp_list)
# file_list += temp_list



# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180912_000000000000.csv')

# random.Random(23).shuffle(temp_list)
# file_list += temp_list



# temp_list = []
# for logN in range(10):
#     temp_list.append('../data/raw/training_set/log_'+str(logN)+'_20180911_000000000000.csv')

# random.Random(23).shuffle(temp_list)
# file_list += temp_list
    

file_list

['../data/raw/training_set/log_7_20180917_000000000000.csv',
 '../data/raw/training_set/log_8_20180917_000000000000.csv',
 '../data/raw/training_set/log_5_20180917_000000000000.csv',
 '../data/raw/training_set/log_6_20180917_000000000000.csv',
 '../data/raw/training_set/log_3_20180917_000000000000.csv',
 '../data/raw/training_set/log_2_20180917_000000000000.csv',
 '../data/raw/training_set/log_9_20180917_000000000000.csv',
 '../data/raw/training_set/log_0_20180917_000000000000.csv',
 '../data/raw/training_set/log_1_20180917_000000000000.csv',
 '../data/raw/training_set/log_4_20180917_000000000000.csv',
 '../data/raw/training_set/log_7_20180916_000000000000.csv',
 '../data/raw/training_set/log_8_20180916_000000000000.csv',
 '../data/raw/training_set/log_5_20180916_000000000000.csv',
 '../data/raw/training_set/log_6_20180916_000000000000.csv',
 '../data/raw/training_set/log_3_20180916_000000000000.csv',
 '../data/raw/training_set/log_2_20180916_000000000000.csv',
 '../data/raw/training_s

In [ ]:
import random
from timeit import default_timer as timer #to see how long the computation will take


nFile = 0
batch_size = 5
while nFile < len(file_list):
    start = timer()
    nFile += batch_size
    df_lookup_list = []
    for file in file_list[(nFile-batch_size):min(nFile, len(file_list))]:
        
        if nFile<=10:
            simFolder = 'similarity_20180917-20180917'
        elif nFile<=20:
            simFolder = 'similarity_20180916-20180917'
        elif nFile<=30:
            simFolder = 'similarity_20180915-20180917'
#         elif nFile<=40:
#             simFolder = 'similarity_20180914-20180917'
#         elif nFile<=50:
#             simFolder = 'similarity_20180913-20180917'
#         elif nFile<=60:
#             simFolder = 'similarity_20180912-20180917'
#         elif nFile<=70:
#             simFolder = 'similarity_20180911-20180917'
            
            
        df_lookup_list.append(mf.prep_dfs(file, tf_df, simFolder, kmean300_df, track6 = True))

    df_lookup = pd.concat(df_lookup_list)
    df_lookup = df_lookup.merge(tf_df.drop(columns = ['key','time_signature','mode']))
    
    # check whether the column names match with the previous training set
    if nFile>batch_size:
        prev_feature_names = lgb.Booster(model_file='../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_incrementalTrain_5_multidays_jan14.txt').feature_name()
        if bool(set(prev_feature_names) - set(df_lookup.columns)): # if there are missing columns
            df_lookup[list(set(prev_feature_names) - set(df_lookup.columns))] = 0 # add the missed columns with 0
            
        if bool(set(df_lookup.columns)- set(prev_feature_names)): # if there are extra columns
            extra_cols = list(set(df_lookup.columns)- set(prev_feature_names) - set(['session_id','track_id_clean','skip_2','weight']))
            df_lookup.drop(columns = extra_cols, inplace = True)
    

    dtrain = lgb.Dataset(df_lookup.drop(columns = ['session_id','track_id_clean','skip_2','weight']).astype('float32'), 
                     label=df_lookup['skip_2'],
                     weight = df_lookup['weight'],
                     free_raw_data=False) # https://lightgbm.readthedocs.io/en/latest/FAQ.html#error-messages-cannot-before-construct-dataset

    
    params = {'num_leaves': 25,
              'learning_rate':0.15,
              'metric': 'binary_error',
              'num_iterations':851,
              'bagging_fraction':0.8,
              'bagging_freq':2,
              'feature_fraction':0.73,
              'min_gain_to_split':0.09,
              'objective': 'binary',
              'force_row_wise': True,
              'num_threads': 5,
              'verbosity': 0,
              'tree_learner': 'voting_parallel'} #https://lightgbm.readthedocs.io/en/latest/Parallel-Learning-Guide.html
    
    if nFile == batch_size:
        bst = lgb.train(params, dtrain)
    else: # continue training on the previous model
        bst = lgb.train(params, dtrain, init_model='../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_incrementalTrain_'+str(int(nFile-batch_size))+'_multidays_jan14.txt')
        
    bst.save_model('../models/SVD/LightGBM_BayesOpt_dec17/for20180918/boost_alltracks_incrementalTrain_'+str(int(nFile))+'_multidays_jan14.txt')

    print('Runtime per batch: %0.2fs' % (timer() - start))